<a href="https://colab.research.google.com/github/iankit3/iankit3/blob/main/distilBERT_seq_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers numpy evaluate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00


In [ ]:
# Clear the Hugging Face cache
import os
os.system("rm -rf ~/.cache/huggingface/datasets")

0

In [ ]:
os.environ["HF_DATASETS_CACHE"] = "/content/hf_cache"
os.environ["HF_DATASETS_OFFLINE"] = "0"  # Ensure online mode
# Set cache directory
os.makedirs("/content/hf_cache", exist_ok=True)

In [ ]:
!pip show datasets
import logging
logging.getLogger("datasets").setLevel(logging.INFO)

Name: datasets
Version: 2.14.4
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, dill, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: evaluate, torchtune


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np

dataset = load_dataset("sst2", download_mode="force_redownload")
train_dataset = dataset['train'].shuffle(seed=42).select(range(2000))
test_dataset = dataset['test'].shuffle(seed=42).select(range(400))

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(data):
    return tokenizer(data["sentence"], padding="max_length", truncation=True, max_length=120)

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)



model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                            #num_labels=2, device_map="auto", attn_implementation="sdpa"
)

training_args = TrainingArguments(
    output_dir="./results_optimized",
    load_best_model_at_end=True,
    eval_strategy='epoch',
    save_strategy='epoch',

    # per_device_train_batch_size=4,
    # per_device_eval_batch_size=4,
    # gradient_accumulation_steps=16,
    # gradient_checkpointing=True,
    # bf16=True,
    # optim="adamw_bnb_8bit",
    # dataloader_pin_memory=True,
    # dataloader_num_workers=2,
    # torch_empty_cache_steps=4,
    # torch_compile_backend="inductor"
)

metrics = evaluate.load("accuracy")
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metrics.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

https://huggingface.co/datasets/sst2/resolve/8d51e7e4887a4caaa95b3fbebbf53c0490b58bbb/README.md not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/81a7e35e22646648efc94186d1db5a77a8d7889e43d38ffd2c831e82524a8988.fa56aaa2f1583718c1c3faf32bde40301a1d422a1162d611dba5a4c7253af72e.incomplete
INFO:datasets.utils.file_utils:https://huggingface.co/datasets/sst2/resolve/8d51e7e4887a4caaa95b3fbebbf53c0490b58bbb/README.md not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/81a7e35e22646648efc94186d1db5a77a8d7889e43d38ffd2c831e82524a8988.fa56aaa2f1583718c1c3faf32bde40301a1d422a1162d611dba5a4c7253af72e.incomplete


storing https://huggingface.co/datasets/sst2/resolve/8d51e7e4887a4caaa95b3fbebbf53c0490b58bbb/README.md in cache at /root/.cache/huggingface/datasets/downloads/81a7e35e22646648efc94186d1db5a77a8d7889e43d38ffd2c831e82524a8988.fa56aaa2f1583718c1c3faf32bde40301a1d422a1162d611dba5a4c7253af72e
INFO:datasets.utils.file_utils:storing https://huggingface.co/datasets/sst2/resolve/8d51e7e4887a4caaa95b3fbebbf53c0490b58bbb/README.md in cache at /root/.cache/huggingface/datasets/downloads/81a7e35e22646648efc94186d1db5a77a8d7889e43d38ffd2c831e82524a8988.fa56aaa2f1583718c1c3faf32bde40301a1d422a1162d611dba5a4c7253af72e
creating metadata file for /root/.cache/huggingface/datasets/downloads/81a7e35e22646648efc94186d1db5a77a8d7889e43d38ffd2c831e82524a8988.fa56aaa2f1583718c1c3faf32bde40301a1d422a1162d611dba5a4c7253af72e
INFO:datasets.utils.file_utils:creating metadata file for /root/.cache/huggingface/datasets/downloads/81a7e35e22646648efc94186d1db5a77a8d7889e43d38ffd2c831e82524a8988.fa56aaa2f1583718c1c3f

hf://datasets/sst2@8d51e7e4887a4caaa95b3fbebbf53c0490b58bbb/data/test-00000-of-00001.parquet not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/0aea38cc5d0f6a06bad169a08fd44278cadb6c06543db89c80581abc4e9db898.incomplete
INFO:datasets.utils.file_utils:hf://datasets/sst2@8d51e7e4887a4caaa95b3fbebbf53c0490b58bbb/data/test-00000-of-00001.parquet not found in cache or force_download set to True, downloading to /root/.cache/huggingface/datasets/downloads/0aea38cc5d0f6a06bad169a08fd44278cadb6c06543db89c80581abc4e9db898.incomplete

storing hf://datasets/sst2@8d51e7e4887a4caaa95b3fbebbf53c0490b58bbb/data/test-00000-of-00001.parquet in cache at /root/.cache/huggingface/datasets/downloads/0aea38cc5d0f6a06bad169a08fd44278cadb6c06543db89c80581abc4e9db898
INFO:datasets.utils.file_utils:storing hf://datasets/sst2@8d51e7e4887a4caaa95b3fbebbf53c0490b58bbb/data/test-00000-of-00001.parquet in cache at /root/.cache/huggingface/datasets/downloads/0a

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split
INFO:datasets.builder:Generating test split


...

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split
INFO:datasets.builder:Generating validation split


Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

All the splits matched successfully.
INFO:datasets.utils.info_utils:All the splits matched successfully.
Dataset sst2 downloaded and prepared to file:///root/.cache/huggingface/datasets/sst2/default/0.0.0/bf2dbdcbc5259c9c. Subsequent calls will reuse this data.
INFO:datasets.builder:Dataset sst2 downloaded and prepared to file:///root/.cache/huggingface/datasets/sst2/default/0.0.0/bf2dbdcbc5259c9c. Subsequent calls will reuse this data.


NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.